# Project 4 - Group 14

### Setup

In [ ]:
#imported libraries go here
import pandas as pd
import seaborn
from collections import Counter
import numpy
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
import re


### Preprocessing

In [ ]:


def corpus_size(lines):
    custom_tokenizer(lines)
    print('____Corpus size____\n\nWords : ',tokens,'\nCharacters: ',characters,'\nVocabulary size: ',len(based))    

with open('../Data/Emotion/train_text.txt',"r", encoding="utf8") as file:
    lines = file.readlines()
# lines = open("").readlines()
def custom_tokenizer(lines):
    global based
    global characters
    global tokens
    #read the data
    with open(lines,"r", encoding="utf8") as file:
        lines = file.readlines()
    #Initialize 2 lists, one containing all real words, as they are presented in the data, another with all words lowercase
    all_words = []
    based_words = []
    characters = 0 #Used to keep track of the total number of characters in data
    tokens = 0 # Used to keep track of the total number of tokens in data
    #for every tweet in the data... 
    for i in lines:
        characters += len(i)
        #find all "words" in the data 
        unfiltered_tokens = re.findall(r"\w+(?:'\w+)?|[^\w\s]",i) 
        #Initialize a list containing all "non-words" tokens
        non_words = []
        for g in range(len(unfiltered_tokens)):
            #if token is specifically: ' append to unfiltered (had problems with apostrophe)
            if bool(re.match(r'\'',unfiltered_tokens[g])):
                non_words.append(unfiltered_tokens[g])
            #if token is not 
            elif not bool(re.match('^[a-zA-Z0-9\']+$',unfiltered_tokens[g])):
                non_words.append(unfiltered_tokens[g])
        #initialize list of all words lowered, if they are not present in non_words
        words = [x.lower() for x in unfiltered_tokens if x not in non_words and x !="user"]
        based_words.extend(words)
        words = ' '.join(words)
        all_words.append(words)
        tokens += len(re.findall(r"\w+(?:'\w+)?|[^\w\s]",i))
    based = Counter(based_words)
    return all_words
#Exporting the reuslts of the tokenizer
tokenizer_words_dict = custom_tokenizer("../Data/Hate/train_text.txt")
file1 = open("../Data/Hate/tokenizer_text.txt","w")
file1.writelines(tokenizer_words_dict)
file1.close() 
tokenizer_words_dict = custom_tokenizer("../Data/Emotion/train_text.txt")
file2 = open("../Data/Emotion/tokenizer_text.txt","w")
file2.writelines(tokenizer_words_dict)
file2.close() 

### Characterizing your data

In [ ]:
#Using the teachers code from lecture 2 to plot zipf's law in the plots below with minor modifications
#taken from https://learnit.itu.dk/pluginfile.php/296257/mod_resource/content/1/zipf.py

#Plotting for the hate dataset

#running the tokenizer to get the global variable
not_relevant = custom_tokenizer("../Data/Hate/train_text.txt")
frq = pd.DataFrame(based.most_common(), columns=['token', 'frequency'])
# Index in the sorted list
frq['idx'] = frq.index + 1

# Frequency normalised by corpus size
frq['norm_freq'] = frq.frequency / len(based)

# Cumulative normalised frequency
frq['cumul_frq'] = frq.norm_freq.cumsum()

seaborn.set_theme(style='whitegrid')

# Plot: Cumulative frequency by index, top 10000 tokens
seaborn.relplot(x='idx', y='cumul_frq', data=frq[:10000], kind='line')
plt.show()

In [ ]:
# Plot: Log-log plot for Zipf's law
frq['log_frq'] = numpy.log(frq.frequency)
frq['log_rank'] = numpy.log(frq.frequency.rank(ascending=False))
seaborn.relplot(x='log_rank', y='log_frq', data=frq)
plt.show()

In [ ]:
#Plotting for the emotion dataset
#running the tokenizer to get the global variable "based"
not_relevant = custom_tokenizer("../Data/Emotion/train_text.txt")
frq = pd.DataFrame(based.most_common(), columns=['token', 'frequency'])

# Index in the sorted list
frq['idx'] = frq.index + 1

# Frequency normalised by corpus size
frq['norm_freq'] = frq.frequency / len(based)

# Cumulative normalised frequency
frq['cumul_frq'] = frq.norm_freq.cumsum()

# Plot: Cumulative frequency by index, top 10000 tokens
seaborn.relplot(x='idx', y='cumul_frq', data=frq, kind='line')
plt.show()

In [ ]:
# Plot: Log-log plot for Zipf's law
frq['log_frq'] = numpy.log(frq.frequency)
frq['log_rank'] = numpy.log(frq.frequency.rank(ascending=False))
seaborn.relplot(x='log_rank', y='log_frq', data=frq)
plt.show()

### Manual Annotation and Inter-Annotation Agreement

In [ ]:
#Cohen's Kappa

def kappa(h):
    h2 = open("../Data/Hate/train_labels.txt", "r").readlines()[:100]
    index = 0
    errors = 0
    a = 0
    b = 0
    c = 0
    d = 0
    print("--- hate ---")
    print("(0:non-hate, 1:hate)")
    for hm, testhm in zip(h,h2):
        index += 1
        if hm[0] != testhm[0]:
            errors += 1
            print("line:", index, "  error:", hm[0], ":", testhm[0])
        if hm[0] == '1':
            if testhm[0] == '1':
                a += 1
            elif testhm[0] == '0':
                b += 1
        elif hm[0] == '0':
            if testhm[0] == '1':
                c += 1
            elif testhm[0] == '0':
                d += 1
    p0 = (a+d)/(a+b+c+d)
    pyes = ((a+b)/(a+b+c+d))*((a+b)/(a+b+c+d))
    pno = ((c+d)/(a+b+c+d))*((b+d)/(a+b+c+d))
    pe = (pyes + pno)
    print("---")
    print("p0: ", p0) #percentage correct
    print("pyes: ", pyes) #probability that both were randomly labeled as hate
    print("pno: ", pno) #probability both were randomly labeled as non-hate
    print("pe: ", pe) #overall random agreement probability 
    print("Agreement: ", ((p0 - pe)/(1-pe))) #inter-rater reliability
    return p0,pe

h1 = open("../Data/Hate/Christian_train_labels.txt", "r").readlines()
h2 = open("../Data/Hate/Cosmin_train_labels.txt", "r").readlines()
h3 = open("../Data/Hate/Daniel_train_labels.txt", "r").readlines()
h4 = open("../Data/Hate/Jacob_train_labels.txt", "r").readlines()

print("Christian's")
p0_1,pe_1 = kappa(h1)
print("---------------------")
print("Cosmin's")
p0_2,pe_2 = kappa(h2)
print("---------------------")
print("Daniel's")
p0_3,pe_3 = kappa(h3)
print("---------------------")
print("Jacob's")
p0_4,pe_4 = kappa(h4)

#calculating the average
p0_average = (p0_1+p0_2+p0_3+p0_4)/4
pe_average = (pe_1+pe_2+pe_3+pe_4)/4

print("---------------------")
print("Average score")
print("p0: ", p0_average)
print("pe: ", pe_average)
print("Agreement: ",((p0_average - pe_average)/(1-pe_average)))

### Classifier

Hate speech dataset using CountVectorizer

In [ ]:
X = custom_tokenizer('../Data/Hate/train_text.txt')
with open("../Data/Hate/train_labels.txt", "r") as fd:
    y_train = fd.read().splitlines()
with open("../Data/Hate/val_labels.txt", "r") as fd:
    y_val = fd.read().splitlines()
tokenizer_val_set = custom_tokenizer("../Data/Hate/val_text.txt")
#Using 932 features for max features since the test set can support just 932 features and the classifier needs the same number of features
#for the training and validation set
CV = CountVectorizer(max_features=932, min_df=5, max_df=0.7,analyzer='word', ngram_range=(1, 3))
x1_train = CV.fit_transform(X)
clf_mlp = MLPClassifier(random_state=1, max_iter=1000).fit(x1_train, y_train)
classifier_rf = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier_sgd = SGDClassifier(alpha=0.00001,loss = "perceptron")
clf_sgd= classifier_sgd.fit(x1_train,y_train)
rf = classifier_rf.fit(x1_train, y_train) 

In [ ]:

CV2 = CountVectorizer(max_features=932, min_df=5, max_df=0.7,analyzer='word', ngram_range=(1, 3))
x1_test = CV2.fit_transform(tokenizer_val_set)
y_pred = rf.predict(x1_test)
y_pred1 = clf_sgd.predict(x1_test)
y_pred_mlp= clf_mlp.predict(x1_test)


In [ ]:
len(CV2.get_feature_names())

In [ ]:


print(confusion_matrix(y_val,y_pred))
print(classification_report(y_val,y_pred))
print(accuracy_score(y_val, y_pred))
print(confusion_matrix(y_val,y_pred1))
print(classification_report(y_val,y_pred1))
print(accuracy_score(y_val, y_pred1))
print(confusion_matrix(y_val,y_pred_mlp))
print(classification_report(y_val,y_pred_mlp))
print(accuracy_score(y_val, y_pred_mlp))

In [ ]:


# with open("../Data/Hate/train_labels.txt", "r") as fd:
#     y_train = fd.read().splitlines()
tfidfconverter = TfidfVectorizer(max_features=911, min_df=5, max_df=0.7,analyzer = 'word',ngram_range=(1, 2))
X_train = tfidfconverter.fit_transform(X).toarray()

classifier_sgd = SGDClassifier(alpha=0.00001,loss = "perceptron")
classifier_rf = RandomForestClassifier(n_estimators=1000, random_state=0)
clf_mlp = MLPClassifier(random_state=1, max_iter=1000).fit(X_train, y_train)
rf2  = classifier_rf.fit(X_train, y_train)
sgd2 = classifier_sgd.fit(X_train,y_train) 

In [ ]:

# tokenizer_words_dict2 = custom_tokenizer("../Data/Hate/val_text.txt")
tfidfconverter2 = TfidfVectorizer(max_features=911, min_df=5, max_df=0.7,analyzer = 'word',ngram_range=(1, 2))
X_test = tfidfconverter2.fit_transform(tokenizer_val_set).toarray()

y_pred2 = rf2.predict(X_test)
y_pred3 = sgd2.predict(X_test)
y_pred_mlp_val = clf_mlp.predict(X_test)

In [ ]:
len(tfidfconverter2.get_feature_names())

In [ ]:
# with open("../Data/Hate/val_labels.txt", "r") as fd:
#     y_test = fd.read().splitlines()
print(confusion_matrix(y_val,y_pred2))
print(classification_report(y_val,y_pred2))
print(accuracy_score(y_val, y_pred2))
print(confusion_matrix(y_val,y_pred3))
print(classification_report(y_val,y_pred3))
print(accuracy_score(y_val, y_pred3))
print(confusion_matrix(y_val,y_pred_mlp_val))
print(classification_report(y_val,y_pred_mlp_val))
print(accuracy_score(y_val, y_pred_mlp_val))

Emotion dataset,Count Vectorizer,validation test

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier

with open("../Data/Emotion/train_labels.txt", "r") as fd:
    y_train_emotion = fd.read().splitlines()
X_emotion = custom_tokenizer("../Data/Emotion/train_text.txt")
tokenizer_emotion_val_set = custom_tokenizer("../Data/Emotion/val_text.txt")
with open("../Data/Emotion/val_labels.txt", "r") as fd:
    y_val_emotion = fd.read().splitlines()

CV = CountVectorizer(max_features=405, min_df=3, max_df=0.8,analyzer='word', ngram_range=(1, 3))
x1_train_emotion = CV.fit_transform(X_emotion)

clf_mlp = MLPClassifier(random_state=1, max_iter=1000).fit(x1_train_emotion, y_train_emotion)
classifier_rf = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier_sgd = SGDClassifier(alpha=0.00001,loss = "log")
clf_sgd= classifier_sgd.fit(x1_train_emotion,y_train_emotion)
rf = classifier_rf.fit(x1_train_emotion, y_train_emotion) 

In [ ]:

CV2 = CountVectorizer(max_features=405, min_df=3, max_df=0.8,analyzer='word', ngram_range=(1, 3))
x1_test = CV2.fit_transform(tokenizer_emotion_val_set)
y_pred = rf.predict(x1_test)
y_pred1 = clf_sgd.predict(x1_test)
y_pred_mlpx= clf_mlp.predict(x1_test)

In [ ]:
len(CV2.get_feature_names())

In [ ]:


print(confusion_matrix( y_val_emotion,y_pred))
print(classification_report( y_val_emotion,y_pred))
print(accuracy_score( y_val_emotion, y_pred))
print(confusion_matrix( y_val_emotion,y_pred1))
print(classification_report( y_val_emotion,y_pred1))
print(accuracy_score( y_val_emotion, y_pred1))
print(confusion_matrix( y_val_emotion,y_pred_mlpx))
print(classification_report( y_val_emotion,y_pred_mlpx))
print(accuracy_score( y_val_emotion, y_pred_mlpx))

In [ ]:

tfidfconverter = TfidfVectorizer(max_features=405, min_df=3, max_df=0.8,analyzer = 'word',ngram_range=(1, 3))
X_train_emotion = tfidfconverter.fit_transform(X_emotion).toarray()

classifier_sgd = SGDClassifier(alpha=0.00001,loss = "perceptron")
classifier_rf = RandomForestClassifier(n_estimators=1000, random_state=0,criterion="entropy")
clf_mlp = MLPClassifier(random_state=1, max_iter=1000).fit(X_train_emotion, y_train_emotion)
rf2  = classifier_rf.fit(X_train_emotion, y_train_emotion)
sgd2 = classifier_sgd.fit(X_train_emotion,y_train_emotion) 

In [ ]:

tfidfconverter2 = TfidfVectorizer(max_features=405, min_df=3, max_df=0.8,analyzer = 'word',ngram_range=(1, 3))
X_test = tfidfconverter2.fit_transform(tokenizer_emotion_val_set).toarray()

y_pred2 = rf2.predict(X_test)
y_pred3 = sgd2.predict(X_test)
y_pred4 = clf_mlp.predict(X_test)

In [ ]:
len(tfidfconverter2.get_feature_names())

In [ ]:

print(confusion_matrix(y_val_emotion,y_pred2))
print(classification_report(y_val_emotion,y_pred2))
print(accuracy_score(y_val_emotion, y_pred2))
print(confusion_matrix(y_val_emotion,y_pred3))
print(classification_report(y_val_emotion,y_pred3))
print(accuracy_score(y_val_emotion, y_pred3))
print(confusion_matrix(y_val_emotion,y_pred4))
print(classification_report(y_val_emotion,y_pred4))
print(accuracy_score(y_val_emotion, y_pred4))

Test set

In [ ]:



with open("../Data/Hate/train_labels.txt", "r") as fd:
    y_train = fd.read().splitlines()
tokenizer_test_set_hate = custom_tokenizer("../Data/Hate/test_text.txt")
with open("../Data/Hate/test_labels.txt", "r") as fd:
    y_test = fd.read().splitlines()

CV = CountVectorizer(max_features=2500, min_df=5, max_df=0.7,analyzer='word', ngram_range=(1, 2))
x1_train = CV.fit_transform(X)
clf_mlp = MLPClassifier(random_state=1, max_iter=1000).fit(x1_train, y_train)
classifier_rf = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier_sgd = SGDClassifier(alpha=0.0001,loss = "log")
clf_sgd= classifier_sgd.fit(x1_train,y_train)
rf = classifier_rf.fit(x1_train, y_train) 

In [ ]:


CV2 = CountVectorizer(max_features=2500, min_df=5, max_df=0.7,analyzer='word', ngram_range=(1, 2))
x1_test = CV2.fit_transform(tokenizer_test_set_hate)
y_pred = rf.predict(x1_test)
y_pred1 = clf_sgd.predict(x1_test)
y_pred_mlp= clf_mlp.predict(x1_test)


In [ ]:

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test,y_pred1))
print(classification_report(y_test,y_pred1))
print(accuracy_score(y_test, y_pred1))
print(confusion_matrix(y_test,y_pred_mlp))
print(classification_report(y_test,y_pred_mlp))
print(accuracy_score(y_test, y_pred_mlp))

In [ ]:

tfidfconverter = TfidfVectorizer(max_features=2541, min_df=5, max_df=0.7,analyzer = 'word',ngram_range=(1, 2))
X_train = tfidfconverter.fit_transform(X).toarray()

classifier_sgd = SGDClassifier(alpha=0.001,loss = "log")
classifier_rf = RandomForestClassifier(n_estimators=1000, random_state=0)
clf_mlp = MLPClassifier(random_state=1, max_iter=1000).fit(X_train, y_train)
rf2  = classifier_rf.fit(X_train, y_train)
sgd2 = classifier_sgd.fit(X_train,y_train) 

In [ ]:

tokenizer_words_dict2 = custom_tokenizer("../Data/Hate/test_text.txt")
tfidfconverter2 = TfidfVectorizer(max_features=2541,min_df=5, max_df=0.7,analyzer = 'word',ngram_range=(1, 2))
X_test = tfidfconverter2.fit_transform(tokenizer_words_dict2).toarray()

y_pred2 = rf2.predict(X_test)
y_pred3 = sgd2.predict(X_test)
y_pred_mlp = clf_mlp.predict(X_test)

In [ ]:
len(tfidfconverter2.get_feature_names())

In [ ]:
with open("../Data/Hate/test_labels.txt", "r") as fd:
    y_test = fd.read().splitlines()
print(confusion_matrix(y_test,y_pred2))
print(classification_report(y_test,y_pred2))
print(accuracy_score(y_test, y_pred2))
print(confusion_matrix(y_test,y_pred3))
print(classification_report(y_test,y_pred3))
print(accuracy_score(y_test, y_pred3))
print(confusion_matrix(y_test,y_pred_mlp))
print(classification_report(y_test,y_pred_mlp))
print(accuracy_score(y_test, y_pred_mlp))

Emotion dataset,test set 

In [ ]:
tokenizer_emotion_test_set = custom_tokenizer("../Data/Emotion/test_text.txt")
with open("../Data/Emotion/test_labels.txt", "r") as fd:
    y_test_emotion = fd.read().splitlines()

CV = CountVectorizer(max_features=1657, min_df=3, max_df=0.8,analyzer='word', ngram_range=(1, 4))
x1_train = CV.fit_transform(X_emotion)
clf_mlp = MLPClassifier(random_state=1, max_iter=1000).fit(x1_train, y_train_emotion)
classifier_rf = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier_sgd = SGDClassifier(alpha=0.00001,loss = "perceptron")
clf_sgd= classifier_sgd.fit(x1_train,y_train_emotion)
rf = classifier_rf.fit(x1_train, y_train_emotion) 

In [ ]:

CV2 = CountVectorizer(max_features=1657, min_df=3, max_df=0.8,analyzer='word', ngram_range=(1, 4))
x1_test = CV2.fit_transform(tokenizer_emotion_test_set)
y_pred = rf.predict(x1_test)
y_pred1 = clf_sgd.predict(x1_test)
y_pred_mlp2= clf_mlp.predict(x1_test)

In [ ]:
len(CV2.get_feature_names())

In [ ]:

print(confusion_matrix(y_test_emotion,y_pred))
print(classification_report(y_test_emotion,y_pred))
print(accuracy_score(y_test_emotion, y_pred))
print(confusion_matrix(y_test_emotion,y_pred1))
print(classification_report(y_test_emotion,y_pred1))
print(accuracy_score(y_test_emotion, y_pred1))
print(confusion_matrix(y_test_emotion,y_pred1))
print(classification_report(y_test_emotion,y_pred_mlp2))
print(accuracy_score(y_test_emotion,y_pred_mlp2))

In [ ]:



tfidfconverter = TfidfVectorizer(max_features=1657,min_df=5, max_df=0.8,analyzer = 'word',ngram_range=(1, 3))
X_train = tfidfconverter.fit_transform(X_emotion).toarray()

classifier_sgd = SGDClassifier(alpha=0.00001,loss = "perceptron")
classifier_rf = RandomForestClassifier(n_estimators=1000, random_state=0,criterion="entropy")
clf_mlp = MLPClassifier(random_state=1, max_iter=1000).fit(X_train, y_train_emotion)
rf2  = classifier_rf.fit(X_train, y_train_emotion)
sgd2 = classifier_sgd.fit(X_train,y_train_emotion) 

In [ ]:

tfidfconverter2 = TfidfVectorizer(max_features=1657,min_df=3, max_df=0.8,analyzer = 'word',ngram_range=(1, 4))
X_test = tfidfconverter2.fit_transform(tokenizer_emotion_test_set).toarray()

y_pred2 = rf2.predict(X_test)
y_pred3 = sgd2.predict(X_test)
y_pred4 = clf_mlp.predict(X_test)

In [ ]:
len(tfidfconverter2.get_feature_names())

In [ ]:

print(confusion_matrix(y_test_emotion,y_pred2))
print(classification_report(y_test_emotion,y_pred2))
print(accuracy_score(y_test_emotion, y_pred2))
print(confusion_matrix(y_test_emotion,y_pred3))
print(classification_report(y_test_emotion,y_pred3))
print(accuracy_score(y_test_emotion, y_pred3))
print(confusion_matrix(y_test_emotion,y_pred4))
print(classification_report(y_test_emotion,y_pred4))
print(accuracy_score(y_test_emotion, y_pred4))